## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

### Data Load

In [2]:
# 데이터 로드
DI_train = pd.read_csv('../data/DI_train_dataset_28_1.csv')
DI_test = pd.read_csv('../data/DI_test_dataset_28_1.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
DI_X = DI_train.drop(['임신_성공_여부', 'ID'], axis=1)
DI_y = DI_train['임신_성공_여부']

In [4]:
print(f"DI_X shape: {DI_X.shape}")
print(f"DI_test shape: {DI_test.drop('ID', axis=1).shape}")

DI_X shape: (6289, 25)
DI_test shape: (2176, 25)


### 인코딩 

In [5]:
DI_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "특정_시술_유형",
    "정자_기증자_나이"
]

In [6]:
# 모든 범주형 변수를 문자열로 변환
DI_X[DI_categorical_columns] = DI_X[DI_categorical_columns].astype(str)
DI_test[DI_categorical_columns] = DI_test[DI_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
DI_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

DI_X[DI_categorical_columns] = DI_encoder.fit_transform(DI_X[DI_categorical_columns])
DI_test[DI_categorical_columns] = DI_encoder.transform(DI_test[DI_categorical_columns])

## Modeling

In [7]:
# 데이터 분할
DI_X_train, DI_X_test, DI_y_train, DI_y_test = train_test_split(DI_X, DI_y, test_size=0.2, random_state=42)

### DI 데이터

In [8]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 9000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 2048),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'subsample': trial.suggest_float('subsample', 0.2, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),

        'random_state': 42,
        'scale_pos_weight': 6.757,
        'objective': 'binary',
        'metric': 'auc',
        'verbose': -1     
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(DI_X_train, DI_y_train)
    
    y_pred_proba = model.predict_proba(DI_X_test)[:, 1]
    
    auc = roc_auc_score(DI_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=4000)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-13 23:36:45,497] A new study created in memory with name: no-name-9180ccc6-917c-418d-8a7e-1bc8b9c05f54
[I 2025-02-13 23:36:48,547] Trial 0 finished with value: 0.604949611138771 and parameters: {'n_estimators': 854, 'learning_rate': 0.09536139125354806, 'num_leaves': 1693, 'min_child_samples': 18, 'subsample': 0.49085020991279515, 'reg_alpha': 0.021048141852539948, 'reg_lambda': 0.10072063416897774}. Best is trial 0 with value: 0.604949611138771.
[I 2025-02-13 23:36:49,195] Trial 1 finished with value: 0.6916121574215552 and parameters: {'n_estimators': 2242, 'learning_rate': 9.213320332448551e-05, 'num_leaves': 251, 'min_child_samples': 24, 'subsample': 0.3122210783470032, 'reg_alpha': 8.087273596150865, 'reg_lambda': 0.6329213785648717}. Best is trial 1 with value: 0.6916121574215552.
[I 2025-02-13 23:36:53,616] Trial 2 finished with value: 0.6164180129448726 and parameters: {'n_estimators': 3910, 'learning_rate': 0.012391724785439847, 'num_leaves': 65, 'min_child_samples'

Best trial:
  Value: 0.7241603287150696
  Params: 
    n_estimators: 839
    learning_rate: 2.3489804762505677e-05
    num_leaves: 1918
    min_child_samples: 1
    subsample: 0.5903310089953149
    reg_alpha: 1.6831295920793434
    reg_lambda: 0.5486596072585076


.